# Chat Models

## 1_chat_model_basic

In [2]:
%pip install -q python-dotenv langchain langchain-community langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv

In [4]:
# Load environment variables from .env
load_dotenv()

True

In [5]:
from langchain_ollama.llms import OllamaLLM

In [6]:
# Create a OllamaLLM model
model = OllamaLLM(base_url="http://localhost:11434", model="llama3.2", temperature=0)

In [23]:
# Invoke the model with a message
result = model.invoke("What is 81 divided by 9?")

In [24]:
print("Full result:")
print(result)

Full result:
81 ÷ 9 = 9


## 2_chat_model_basic_conversation

In [25]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [26]:
from langchain_ollama import ChatOllama

In [27]:
model = ChatOllama(
        base_url="http://localhost:11434",
        model="llama3.2",
        temperature=0)

In [28]:
# SystemMessage:
#   Message for priming AI behavior, usually passed in as the first of a sequenc of input messages.
# HumanMessagse:
#   Message from a human to the AI model.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

In [29]:
# Invoke the model with messages
result = model.invoke(messages)

In [30]:
print(f"Answer from AI: {result}")

Answer from AI: content='81 ÷ 9 = 9' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-10T22:08:45.102166555Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1986353348, 'load_duration': 22661379, 'prompt_eval_count': 40, 'prompt_eval_duration': 1169000000, 'eval_count': 9, 'eval_duration': 793000000} id='run-5d386072-90ea-4b12-8615-1448787f8c65-0' usage_metadata={'input_tokens': 40, 'output_tokens': 9, 'total_tokens': 49}


In [31]:
print(f"Answer from AI: {result.content}")

Answer from AI: 81 ÷ 9 = 9


In [32]:
# AIMessage:
#   Message from an AI.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
]

In [33]:
# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

Answer from AI: 10 × 5 = 50


In [34]:
# Invoke the model with messages
result = model.invoke("Summarize our conversation so far")
print(f"Answer from AI: {result.content}")

Answer from AI: We haven't had a conversation yet. This is the first message in our conversation. What would you like to talk about? I can summarize our conversation at the end if you'd like.


In [35]:
# AIMessage:
#   Message from an Human.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
    AIMessage(content="10 times 5 is 50."),
    HumanMessage(content="Summarize our conversation so far")
]

In [36]:
# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

Answer from AI: We've had a simple math conversation so far. You asked me two questions:

1. To calculate the result of dividing 81 by 9, which I answered as 9.
2. To calculate the result of multiplying 10 by 5, which I answered as 50.

That's the summary of our conversation so far!


### 3_chat_model_alternatives

In [37]:
%pip install -q langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [39]:
openai_model = ChatOpenAI(model="gpt-4o-mini")

In [40]:
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

In [41]:
# Invoke the model with messages
result = openai_model.invoke(messages)
print(f"Answer from OpenAI: {result.content}")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ???. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## 4_chat_model_conversation_with_user

In [42]:
# Use a list to store messages
chat_history = []

In [43]:
# Set an initial system message (optional)
system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)  # Add system message to chat history

In [44]:
# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))  # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))  # Add AI message

    print(f"AI: {response}")


print("---- Message History ----")
print(chat_history)

AI: Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?
---- Message History ----
[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={})]


## 5_chat_model_conversation_memory

In [45]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [46]:
# store is a dictionary that maps session IDs to their corresponding chat histories.
store = {}  # memory is maintained outside the chain

In [47]:
# A function that returns the chat history for a given session ID.
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [48]:
#  Define a RunnableConfig object, with a `configurable` key. session_id determines thread
config = {"configurable": {"session_id": "1"}}

In [49]:
conversation = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [50]:
result = conversation.invoke(
    "Hi I'm Srijan Dubey.",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Hello Srijan! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [51]:
result = conversation.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Your name is Srijan Dubey, as per the information provided earlier.


In [52]:
store

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi I'm Srijan Dubey.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Srijan! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-10T22:10:33.494668852Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3166129191, 'load_duration': 21628889, 'prompt_eval_count': 34, 'prompt_eval_duration': 706000000, 'eval_count': 28, 'eval_duration': 2437000000}, id='run-12ba99bd-aa60-4dad-b175-b5012631da5c-0', usage_metadata={'input_tokens': 34, 'output_tokens': 28, 'total_tokens': 62}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Srijan Dubey, as per the information provided earlier.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '20

In [53]:
%pip install -q langchain-community SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [54]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [83]:
# Function to get session history
# create sync sql message history by connection_string
def get_session_history(session_id: str):
    return SQLChatMessageHistory(
        session_id=session_id, 
        connection="sqlite:///memory.db"
    )

In [84]:
config = {"configurable": {"session_id": "session1"}}

In [85]:
conversation_sql = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [86]:
result = conversation_sql.invoke(
    "Hi I'm Srijan Dubey.",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Hello Srijan! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [48]:
result = conversation_sql.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Your name is Srijan Dubey. We had established that earlier in our conversation. Is there something else I can help you with or would you like to talk about something specific?


In [49]:
config = {"configurable": {"session_id": "session2"}}

In [50]:
result = conversation_sql.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: I don't have any information about your identity or personal details, including your name. I'm a large language model, I don't retain any data about individual users, and our conversation just started. If you'd like to share your name with me, I'd be happy to chat with you!


In [51]:
# create sync sql message history by connection_string
message_history = get_session_history("session1")

In [52]:
message_history.get_messages()

[HumanMessage(content="Hi I'm Srijan Dubey.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Srijan! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-09T22:46:22.605161136Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2306479174, 'load_duration': 18936601, 'prompt_eval_count': 34, 'prompt_eval_duration': 77000000, 'eval_count': 28, 'eval_duration': 2209000000}, id='run-4058ca45-85f5-44c1-9879-e3941c72c217-0', usage_metadata={'input_tokens': 34, 'output_tokens': 28, 'total_tokens': 62}),
 HumanMessage(content="Hi I'm Srijan Dubey.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Nice to meet you again, Srijan! How's your day going so far?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-09T22:49:38.220503497Z', 'message': {